<a href="https://colab.research.google.com/github/prathameshk30/Kaggle-competition-CommonLit-Evaluate-Student-Summaries/blob/main/competition_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goal of the Competition
#### The goal of this competition is to assess the quality of summaries written by students in grades 3-12. You'll build a model that evaluates how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary. You'll have access to a collection of real student summaries to train your model.

#### Your work will assist teachers in evaluating the quality of student work and also help learning platforms provide immediate feedback to students.

## Link to competition

https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries (July 2023- Oct 2023)

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00


In [2]:
#Importing the libraries
import warnings
warnings.filterwarnings('ignore')
import os
import random
import time

import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
import xgboost
from xgboost import XGBRFRegressor
import transformers
from sklearn.model_selection import KFold, GroupKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

In [3]:
path='/content/'

In [4]:
# Reading the dataset

df_summ_tr=pd.read_csv(path+r'summaries_train.csv')
df_summ_tr

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [5]:
df_prompts_tr=pd.read_csv(path+r'prompts_train.csv')
df_prompts_tr

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [6]:
df=df_summ_tr.join(df_prompts_tr.set_index('prompt_id'), on='prompt_id')
df

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [7]:
pd.DataFrame(df.prompt_question.value_counts())

,prompt_question
"Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.",2057
"In complete sentences, summarize the structure of the ancient Egyptian system of government. How were different social classes involved in this government? Cite evidence from the text.",2009
Summarize the various ways the factory would use or cover up spoiled meat. Cite evidence in your answer.,1996
Summarize how the Third Wave developed over such a short period of time and why the experiment was ended.,1103


In [8]:
df_summ_ts=pd.read_csv(path+r'summaries_test.csv')
df_summ_ts

,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


In [9]:
df_prompts_ts=pd.read_csv(path+r'prompts_test.csv')
df_prompts_ts

,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


### Using TD-IDF

In [10]:
# We start by text preprocessing on the dataframe
def preprocessText(text):
    try:
        # replace newline with space
        text = text.replace("\n", " ")
        # split text
        words = text.split()
        # convert to lowercase
        words = [word.lower() for word in words]
        # remove punctuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in words]
        # select only alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        # stop word removal
        stop_words = spacy.lang.en.stop_words.STOP_WORDS
        words = [w for w in words if not w in stop_words]
        # return pre-processed paragraph text
        text = ' '.join(words)
        return text
    except:
        return text

In [11]:
# Create a new Dataframe with cleaned text

df_clean=pd.DataFrame()
df_clean["prompt_title"] = df["prompt_title"].apply(lambda x: preprocessText(x))
df_clean['prompt_question']= df["prompt_question"].apply(lambda x: preprocessText(x))
df_clean['prompt_text']= df["prompt_text"].apply(lambda x: preprocessText(x))
df_clean['text']= df["text"].apply(lambda x: preprocessText(x))
df_clean['content']=df['content']
df_clean['wording']=df['wording']
df_clean

,prompt_title,prompt_question,prompt_text,text,content,wording
0,wave,summarize wave developed short period time exp...,background wave experiment took place cubberle...,wave experimentto people reacted new leader go...,0.205683,0.380538
1,excerpt jungle,summarize ways factory use cover spoiled meat ...,member trimming beef cannery working sausage f...,rub soda smell away wouldnt bad smell meat tos...,-0.548304,0.506755
2,egyptian social structure,complete sentences summarize structure ancient...,egyptian society structured like pyramid gods ...,egypt occupations social classes involved dayt...,3.128928,4.231226
3,egyptian social structure,complete sentences summarize structure ancient...,egyptian society structured like pyramid gods ...,highest class pharaohs people godsthen highest...,-0.210614,-0.471415
4,wave,summarize wave developed short period time exp...,background wave experiment took place cubberle...,wave developed rapidly students genuinly belie...,3.272894,3.219757
...,...,...,...,...,...,...
7160,excerpt jungle,summarize ways factory use cover spoiled meat ...,member trimming beef cannery working sausage f...,sorts chemical concoctions meat fine shown quo...,0.205683,0.380538
7161,egyptian social structure,complete sentences summarize structure ancient...,egyptian society structured like pyramid gods ...,lowest classes slaves farmers slaves people ta...,-0.308448,0.048171
7162,egyptian social structure,complete sentences summarize structure ancient...,egyptian society structured like pyramid gods ...,sorta people start working structour theyed ba...,-1.408180,-0.493603
7163,tragedy,summarize elements ideal tragedy described ari...,chapter sequel said proceed consider poet aim ...,ideal tragety elements ideal start great trage...,-0.393310,0.627128


In [13]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
idf_promt_text=tf_idf.fit_transform(df_clean['prompt_text'])
idf_text=tf_idf.fit_transform(df_clean['text'])
idf_promt_question=tf_idf.fit_transform(df_clean['prompt_question'])
idf_promt_title=tf_idf.fit_transform(df_clean['prompt_title'])

# function to connvert to dataframe
def convert_to_df(df):
  df=pd.DataFrame(df.todense())
  return df

#convert to pandas df
idf_promt_text=convert_to_df(idf_promt_text)
idf_text=convert_to_df(idf_text)
idf_promt_question=convert_to_df(idf_promt_question)
idf_promt_title=convert_to_df(idf_promt_title)

#joining idf dataframe
df_idf=pd.concat([idf_promt_text,idf_text,idf_promt_question,idf_promt_title], axis=1)

In [ ]:
# Splitting the Dataset in train test

X=df_idf
y=df_clean[['content', 'wording']]

### Create the training class

In [12]:
class train_model:
  def __init__(self, X, y, test_size, random_state, Model, loss,*args):
    self.X=X
    self.y=y
    self.test_size=test_size
    self.random_state=random_state
    self.model=Model
    self.loss=loss

  def split_dataset(self):
    X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.test_size, random_state=self.random_state)
    return X_train, X_test, y_train, y_test

  def train_predict_model(self):
    X_train, X_test, y_train, y_test=self.split_dataset()
    model = self.model
    model.fit(X_train, y_train)
    y_hat=model.predict(X_test)
    # Define MSE
    mse = self.loss(y_test, y_hat, multioutput='raw_values')
    # Define MCRMSE
    col_rmse = np.sqrt(np.mean((y_hat - y_test) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)
    print(f'Mean squared error is (RMSE) {mse}'), print(f'Mean column squared error is (MCRMSE) {mcrmse}')

In [ ]:
train_class=train_model(X, y, 0.33 ,42, RandomForestRegressor(),mean_squared_error)
train_class.train_predict_model()

Mean squared error is (RMSE) [0.40582707 0.56285934]
Mean column squared error is (MCRMSE) 0.6936425533422992


(None, None)

### Trying BERT Embeddings

In [13]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [14]:
class transformers_embedding:
  # Defining class parameters
  def __init__(self,model,df):
    self.model=model
    self.df=df

  #Check text columns
  def check_column_type(self):
    return [i for i in self.df.columns if self.df[i].dtypes==object]

  # Load the pre-trained sentence embedding model
  def define_model_config(self):
    model_name = self.model
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    model = transformers.AutoModel.from_pretrained(model_name)
    return tokenizer, model

  # Define a function to compute the sentence embedding
  def sentence_embedding(self,sentence):
    tokenizer, model= self.define_model_config()
    input_ids = torch.tensor(tokenizer.encode(sentence, truncation=True)).unsqueeze(0)
    with torch.no_grad():
        output = model(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

  # Get embeddings
  def get_embedd(self):
    column_name=self.check_column_type()
    embedding_list=[]
    for i in tqdm.tqdm(column_name):
      embedding_list.append([self.sentence_embedding(review) for review in self.df[i]])
    return embedding_list

  #Defining a function to fine tune selected transformer model
  def fine_tune_model(self):
    # Calling Model config
    tokenizer, model= self.define_model_config()
    # Calling only text column for preprocessing
    column_name=self.check_column_type()

    # Saving input ids and attentionmask of sentences
    list_of_input_ids=[]
    list_of_attention_mask=[]

    # Saving the input ids and attention masks for each column (4 text column)
    for i in tqdm.tqdm(column_name):
      input_ids = []
      attention_masks = []
      for sentence in self.df[i]:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 512,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

      # Convert the lists into tensors.
      input_ids = torch.cat(input_ids, dim=0)
      attention_masks = torch.cat(attention_masks, dim=0)

      # Saving the input ids and attention mask to list
      list_of_input_ids.append(input_ids)
      list_of_attention_mask.append(attention_masks)

      #Save labels
      labels=self.df[i].
      labels = torch.tensor(labels)

# Training & Validation Split

    # Combine the training inputs into a TensorDataset.
    dataset = TensorDataset(input_ids, attention_masks, labels)


SyntaxError: ignored

In [ ]:
# Creating embeddings using BERT model

model_config=transformers_embedding("bert-base-uncased", df.drop(columns=['student_id','prompt_id']))
embeddings_list=model_config.get_embedd()
len(embeddings_list)

In [ ]:
#Save embedding
embeddings_list = np.array(embeddings_list)
np.save(path+'embeddings_list.npy', embeddings_list)

In [ ]:
#Load the saved embedding
embeddings_list=np.load(path+'embeddings_list.npy')

In [ ]:
#convert to pandas df for each of the columns
idf_promt_text=pd.DataFrame(embeddings_list[0])
idf_text=pd.DataFrame(embeddings_list[1])
idf_promt_question=pd.DataFrame(embeddings_list[2])
idf_promt_title=pd.DataFrame(embeddings_list[3])

#joining idf dataframe
df_embedd=pd.concat([idf_promt_text,idf_text,idf_promt_question,idf_promt_title], axis=1)

In [ ]:
# Changing columns head

df_embedd.columns=np.arange(len(df_embedd.columns))

In [ ]:
# Splitting the Dataset in train test

X=df_embedd
y=df_clean[['content', 'wording']]

### Use direct BERT base uncased (Without training) to create word embedding.  

In [ ]:
# Use direct BERT model to create embedding and check
train_class=train_model(X, y, 0.33 ,42,RandomForestRegressor(),mean_squared_error)
train_class.train_predict_model()

### Using A fine Tuned BERT model to create word embeddings

In [59]:
class transformers_embedding:
  # Defining class parameters
  def __init__(self ,model ,df, list_of_targets):
    self.model=model
    self.df=df
    self.list_of_targets=(list_of_targets)

  #Check text columns
  def check_column_type(self):
    return [i for i in self.df.columns if self.df[i].dtypes==object]

  # Load the pre-trained sentence embedding model
  def define_model_config(self):
    model_name = self.model
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    model = transformers.AutoModel.from_pretrained(model_name)
    return tokenizer, model

  # Define a function to compute the sentence embedding
  def sentence_embedding(self,sentence):
    tokenizer, model= self.define_model_config()
    input_ids = torch.tensor(tokenizer.encode(sentence, truncation=True)).unsqueeze(0)
    with torch.no_grad():
        output = model(input_ids)
    embeddings = output.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

  # Get embeddings
  def get_embedd(self, epochs, learning_rate):
    column_name=self.check_column_type()
    embedding_list=[]
    for i in column_name:
      embedding_list.append([self.sentence_embedding(review) for review in self.df[i]])
    return embedding_list

  def flat_accuracy(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

  def format_time(self, elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

  #Defining a function to fine tune
  def fine_tune_model(self, learning_rate, epochs):
    # Calling Model config
    tokenizer, model= self.define_model_config()
    # Calling only text column for preprocessing
    column_name=self.check_column_type()

    list_of_input_ids=[]
    list_of_attention_mask=[]

    #Saving the input ids and attention masks for each column
    for i in column_name:
      input_ids_list = []
      attention_masks_list = []
      for sentence in self.df[i]:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 512,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                            truncation=True
                      )

        # Add the encoded sentence to the list.
        input_ids_list.append((encoded_dict['input_ids']))
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_list.append((encoded_dict['attention_mask']))

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids_list, dim=0)
        attention_masks = torch.cat(attention_masks_list, dim=0)

      # Saving the input ids and attention mask to list
      list_of_input_ids.append(input_ids)
      list_of_attention_mask.append(attention_masks)

    # return list_of_input_ids

    # Training model based on target column [wording, content]
    for column in zip(list_of_input_ids,list_of_attention_mask):
      for target in self.list_of_targets:
        # print(target)
        labels=self.df[target]
        labels = torch.tensor(labels)

        # Define input ids and attention masks
        input_ids=column[0]
        attention_masks=column[1]

        # Combine the training inputs into a TensorDataset.
        dataset = TensorDataset(input_ids, attention_masks, labels)

        # Create a 90-10 train-validation split.
        # Calculate the number of samples to include in each set.
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        # The DataLoader needs to know our batch size for training, so we specify it here. For fine-tuning BERT on a specific task, the authors recommend a batch
        # size of 16 or 32.
        batch_size = 32

        # Create the DataLoaders for our training and validation sets.
        # We'll take training samples in random order.
        train_dataloader = DataLoader(
                    train_dataset,  # The training samples.
                    sampler = RandomSampler(train_dataset), # Select batches randomly
                    batch_size = batch_size # Trains with this batch size.
                )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        validation_dataloader = DataLoader(
                    val_dataset, # The validation samples.
                    sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                    batch_size = batch_size # Evaluate with this batch size.
                )
    # return train_dataset

        # Load BertForSequenceClassification, the pretrained BERT model with a single
        # linear classification layer on top.
        model = BertForSequenceClassification.from_pretrained(
            self.model, # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification.
                            # You can increase this for multi-class tasks.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
        )

        # Tell pytorch to run this model on the GPU.
        model.cuda()
        # Note: AdamW is a class from the huggingface library (as opposed to pytorch)
        # I believe the 'W' stands for 'Weight Decay fix"
        optimizer = AdamW(model.parameters(),
                          lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                          eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )

        # Number of training epochs. The BERT authors recommend between 2 and 4.
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        epochs = epochs

        # Total number of training steps is [number of batches] x [number of epochs].
        # (Note that this is not the same as the number of training samples).
        total_steps = len(train_dataloader) * epochs

        # Create the learning rate scheduler.
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

        # Set the seed value all over the place to make this reproducible.
        seed_val = 42

        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)

        # We'll store a number of quantities such as training and validation loss,
        # validation accuracy, and timings.
        training_stats = []

        # Measure the total training time for the whole run.
        total_t0 = time.time()

        # For each epoch...
        for epoch_i in range(0, epochs):

            # ========================================
            #               Training
            # ========================================

            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            model.train()

            # For each batch of training data...
            for step, batch in enumerate(train_dataloader):

                # Progress update every 40 batches.
                if step % 40 == 0 and not step == 0:
                    # Calculate elapsed time in minutes.
                    elapsed = format_time(time.time() - t0)

                    # Report progress.
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

                # Unpack this training batch from our dataloader.
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using the
                # `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids
                #   [1]: attention masks
                #   [2]: labels
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because
                # accumulating the gradients is "convenient while training RNNs".
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                model.zero_grad()

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here:
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided labels) and the "logits"--the model
                # outputs prior to activation.
                loss, logits = model(b_input_ids,
                                    token_type_ids=None,
                                    attention_mask=b_input_mask,
                                    labels=b_labels)

                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value
                # from the tensor.
                total_train_loss += loss.item()

                # Perform a backward pass to calculate the gradients.
                loss.backward()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()

            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len(train_dataloader)

            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)

            print("")
            print("  Average training loss: {0:.2f}".format(avg_train_loss))
            print("  Training epcoh took: {:}".format(training_time))

            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            model.eval()

            # Tracking variables
            total_eval_accuracy = 0
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in validation_dataloader:

                # Unpack this training batch from our dataloader.
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids
                #   [1]: attention masks
                #   [2]: labels
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here:
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    (loss, logits) = model(b_input_ids,
                                          token_type_ids=None,
                                          attention_mask=b_input_mask,
                                          labels=b_labels)

                # Accumulate the validation loss.
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                total_eval_accuracy += flat_accuracy(logits, label_ids)


            # Report the final accuracy for this validation run.
            avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
            print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(validation_dataloader)

            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)

            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Valid. Accur.': avg_val_accuracy,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )

        print("")
        print("Training complete!")

        print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

    pd.set_option('precision', 2)
    # Create a DataFrame from our training statistics.
    df_stats = pd.DataFrame(data=training_stats)

    # Use the 'epoch' as the row index.
    df_stats = df_stats.set_index('epoch')

    # A hack to force the column headers to wrap.
    #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

    # Display the table.
    df_stats

In [60]:
model_config=transformers_embedding("bert-base-uncased", df.drop(columns=['student_id','prompt_id'])[:100],['content', 'wording'])
a=model_config.fine_tune_model(0.0000001,8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 8 ========
Training...


ValueError: ignored

In [58]:
len(a)

90